### Импортируем библиотеки

In [1]:
import csv
import time
import requests
import pandas as pd

### Назначим глобальные переменные

In [2]:
token = 'Введите свой token'
version = 5.131
count = 100

### Напишем функцию парсинга постов на стене ('wall.get')

In [3]:
def take_all_posts():
    domain = '1tsprint'
    all_posts = []
    offset = 0
    while offset < 1000:
        response = requests.get('https://api.vk.com/method/wall.get',
                                params={
                                    'access_token': token,
                                    'v': version,
                                    'domain': domain,
                                    'count': count,
                                    'offset': offset
                                    }
                                )
        data = response.json()['response']['items']
        offset += 100
        all_posts.extend(data)
        time.sleep(0.5)
    return all_posts
all_posts = take_all_posts()

### Соберем таблицу постов за все время, отдельным столбцом 'count_comments' выведем количество комментариев к каждому посту.

In [4]:
list_posts, list_comments, list_date_of_post, list_post_id = [], [], [], []
for post in all_posts:
    list_posts.append(post['text'])
    list_comments.append(post['comments']['count'])
    list_date_of_post.append(post['date'])
    list_post_id.append(post['id'])
list_date_of_post = pd.Series(list_date_of_post)   

count_comment_df = pd.concat([pd.Series(list_post_id), pd.Series(list_posts), pd.Series(list_comments), list_date_of_post.astype('int').astype("datetime64[s]")], axis=1)
count_comment_df.columns = ['post_id', 'text_post', 'count_comments', 'date_public']
count_comment_df.head()

,post_id,text_post,count_comments,date_public
0,560,Что такое Zoom-этикет \n \n💻 Видеоконференции ...,0,2023-07-16 12:45:00
1,559,Мастермайнд: как находить лучшие решения с пом...,0,2023-07-15 12:30:00
2,554,Заключительная часть советов от участников раз...,2,2023-07-14 13:04:05
3,549,Как понимать корпоративный сленг \n \n💬 У нас ...,0,2023-07-13 13:45:00
4,547,Как в 1Т Клубе организована менторская поддерж...,0,2023-07-12 13:00:01


### Посмотрим, сколько всего постов посвящены тематике Data Engineering:

In [5]:
posts_de = count_comment_df['text_post'].str.lower()
posts_de = posts_de[posts_de.str.contains('дата инженер|data eng|инженер данных')]
print(f'Всего найдено {len(posts_de)} постов с тематикой "дата инженер|data eng|инженер данных".')
posts_de

Всего найдено 6 постов с тематикой "дата инженер|data eng|инженер данных".


55     🎙 в эту среду, 24 мая, в 19 часов по мск — оче...
76     🖥 приглашаем на вебинар сегодня в 20:00! \n \n...
82     на нашем youtube-канале вышла запись вебинара ...
84     ⏰ начинаем через полчаса! \n \nвебинар «как по...
87     🚀 три новых курса от 1т по искусственному инте...
322    курс «инженер данных» https://sprint.1t.ru/eng...
Name: text_post, dtype: object

### Выведем отдельную таблицу, где отберем посты, в которых оставлен хотя бы 1 комментарий.

In [6]:
df_with_comment = count_comment_df.loc[count_comment_df['count_comments'] > 0]
df_with_comment

,post_id,text_post,count_comments,date_public
2,554,Заключительная часть советов от участников раз...,2,2023-07-14 13:04:05
13,531,"7 полезных сервисов для тех, кто работает с те...",1,2023-07-01 12:30:00
33,499,Чего не стоит делать на собеседовании \n \nВы ...,1,2023-06-11 12:30:00
38,485,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,3,2023-06-06 12:30:00
100,383,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,3,2023-04-17 13:48:06
123,341,"🏴‍☠ Выложили на нашем YouTube-канале видео, то...",2,2023-03-27 15:43:05
140,309,🔔 Свистать всех наверх! \n \n🏴‍☠ ИТ-конференци...,1,2023-03-11 09:27:45
145,299,Приручаем искусственный интеллект \n \n🎨 1 мар...,5,2023-03-07 15:40:00
190,198,Как работает Bluetooth \n \n⚡ Bluetooth — техн...,1,2023-01-22 13:00:00
211,156,Что умеют нейросети? \n \nНейросети уже стали ...,1,2022-12-28 13:28:39


### Посмотрим, сколько комментариев оставлено к постам с совпадением тематики Data Engineering.

In [7]:
posts_de_com = df_with_comment['text_post'].str.lower()
posts_de_com = posts_de_com[posts_de_com.str.contains('дата инженер|data eng|инженер данных')]
print(f'В списке постов с совпадением тематики "дата инженер|data eng|инженер данных" содержится всего {len(posts_de_com)} комментариев на любую тему.') 

В списке постов с совпадением тематики "дата инженер|data eng|инженер данных" содержится всего 0 комментариев на любую тему.


### Проанализируем,ко скольким постам всего из общего количества пользователи оставили комментарии.

In [8]:
df_post_id_comment = df_with_comment['post_id']
print(f'Всего к {len(df_post_id_comment)} постам из {len(count_comment_df)} посетители оставили комментарии.')

Всего к 16 постам из 323 посетители оставили комментарии.


### На основании анализа выше, соберем таблицу всех комментариев, когда-либо оставленных к постам.
### Для начала соберем json-файл, используя метод 'wall.getComments'.

In [9]:
owner_id = '-215426617'
all_comments = []
offset = 0
count = len(df_post_id_comment)
for post_id in df_post_id_comment:
    response = requests.get('https://api.vk.com/method/wall.getComments',
                            params={
                                'access_token': token,
                                'v': version,
                                'owner_id': owner_id,
                                'count': count,
                                'offset': offset,
                                'post_id': post_id
                                }
                            )
    data = response.json()['response']['items']
    all_comments.extend(data)
all_comments

[{'id': 555,
  'from_id': 30472918,
  'date': 1689340405,
  'text': 'Если ты работать вздумал,\nРазвернись и выпей кофе.\nКто задачи выполняет,\nПо регламенту и в срок? \nПодождёт тебя заказчик, \nПередвинет снова сроки, \nЕсли было так и раньше, \nПроект снова подождёт!',
  'post_id': 554,
  'owner_id': -215426617,
  'parents_stack': [],
  'thread': {'count': 0,
   'items': [],
   'can_post': True,
   'show_reply_button': True}},
 {'id': 556,
  'from_id': 30472918,
  'date': 1689343422,
  'text': 'Если ты командной рулишь, \nМенеджер- руководитель, \nТо задачи ставь команде\nКоротко, и в месяц раз.\nНе планируй сроки сдачи, \nИ по скраму не спринтуйте,\nЛучше сразу попросите\nОплатить скорей проект!',
  'post_id': 554,
  'owner_id': -215426617,
  'parents_stack': [],
  'thread': {'count': 0,
   'items': [],
   'can_post': True,
   'show_reply_button': True}},
 {'id': 536,
  'from_id': 354733385,
  'date': 1688487567,
  'text': 'Классная подборка. Рады, что вы пользуетесь нашим Сервисо

### Потом выведем все полученные данные в таблицу

In [10]:
list_comment_id, list_comment_text, list_comment_date, list_from_id, list_post_id = [], [], [], [], []
for comment in all_comments:
    list_post_id.append(comment['post_id'])
    list_comment_id.append(comment['id'])
    list_comment_date.append(comment['date'])
    list_comment_text.append(comment['text'])
    if comment['from_id'] < 0:
        comment['from_id'] = -(comment['from_id'])
    else: comment['from_id']
    list_from_id.append(comment['from_id'])
list_comment_date = pd.Series(list_comment_date)

all_comments_df = pd.concat([pd.Series(list_post_id), pd.Series(list_comment_id), list_comment_date.astype('int').astype("datetime64[s]"), pd.Series(list_comment_text), pd.Series(list_from_id)], axis=1)
all_comments_df.columns = ['post_id', 'comment_id', 'comment_date', 'comment_text', 'comment_from_id']
all_comments_df

,post_id,comment_id,comment_date,comment_text,comment_from_id
0,554,555,2023-07-14 13:13:25,"Если ты работать вздумал,\nРазвернись и выпей ...",30472918
1,554,556,2023-07-14 14:03:42,"Если ты командной рулишь, \nМенеджер- руководи...",30472918
2,531,536,2023-07-04 16:19:27,"Классная подборка. Рады, что вы пользуетесь на...",354733385
3,499,500,2023-06-11 12:44:51,"Из последнего: \n""- Вы работаете с Битрикс?\n-...",280437919
4,485,486,2023-06-06 12:41:38,"Здравствуйте, вроде 10 мышек разыгрывали, по-м...",387550114
5,485,487,2023-06-06 14:01:35,Большое спасибо 1T Sprint за возможность участ...,752917537
6,485,488,2023-06-06 14:59:33,"Спасибо за конкурс, приятно вспомнить с чего в...",699485614
7,383,384,2023-04-17 16:33:48,,282137570
8,383,385,2023-04-17 16:34:05,,282137570
9,383,386,2023-04-17 19:01:47,Это был лучший офлайн)\nЭто было не реально кл...,25359617


### Подтянем к таблице с комментариями данные о постах по ключу 'post_id'.

In [11]:
df_temp = all_comments_df.merge(df_with_comment, left_on='post_id', right_on='post_id', how='left')
df_temp

,post_id,comment_id,comment_date,comment_text,comment_from_id,text_post,count_comments,date_public
0,554,555,2023-07-14 13:13:25,"Если ты работать вздумал,\nРазвернись и выпей ...",30472918,Заключительная часть советов от участников раз...,2,2023-07-14 13:04:05
1,554,556,2023-07-14 14:03:42,"Если ты командной рулишь, \nМенеджер- руководи...",30472918,Заключительная часть советов от участников раз...,2,2023-07-14 13:04:05
2,531,536,2023-07-04 16:19:27,"Классная подборка. Рады, что вы пользуетесь на...",354733385,"7 полезных сервисов для тех, кто работает с те...",1,2023-07-01 12:30:00
3,499,500,2023-06-11 12:44:51,"Из последнего: \n""- Вы работаете с Битрикс?\n-...",280437919,Чего не стоит делать на собеседовании \n \nВы ...,1,2023-06-11 12:30:00
4,485,486,2023-06-06 12:41:38,"Здравствуйте, вроде 10 мышек разыгрывали, по-м...",387550114,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,3,2023-06-06 12:30:00
5,485,487,2023-06-06 14:01:35,Большое спасибо 1T Sprint за возможность участ...,752917537,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,3,2023-06-06 12:30:00
6,485,488,2023-06-06 14:59:33,"Спасибо за конкурс, приятно вспомнить с чего в...",699485614,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,3,2023-06-06 12:30:00
7,383,384,2023-04-17 16:33:48,,282137570,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,3,2023-04-17 13:48:06
8,383,385,2023-04-17 16:34:05,,282137570,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,3,2023-04-17 13:48:06
9,383,386,2023-04-17 19:01:47,Это был лучший офлайн)\nЭто было не реально кл...,25359617,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,3,2023-04-17 13:48:06


### Поменяем колонки местами для наглядности и лучшего восприятия.

In [12]:
df_all = df_temp[['post_id', 'date_public', 'text_post', 'comment_id', 'comment_date', 'comment_text', 'comment_from_id']]
df_all

,post_id,date_public,text_post,comment_id,comment_date,comment_text,comment_from_id
0,554,2023-07-14 13:04:05,Заключительная часть советов от участников раз...,555,2023-07-14 13:13:25,"Если ты работать вздумал,\nРазвернись и выпей ...",30472918
1,554,2023-07-14 13:04:05,Заключительная часть советов от участников раз...,556,2023-07-14 14:03:42,"Если ты командной рулишь, \nМенеджер- руководи...",30472918
2,531,2023-07-01 12:30:00,"7 полезных сервисов для тех, кто работает с те...",536,2023-07-04 16:19:27,"Классная подборка. Рады, что вы пользуетесь на...",354733385
3,499,2023-06-11 12:30:00,Чего не стоит делать на собеседовании \n \nВы ...,500,2023-06-11 12:44:51,"Из последнего: \n""- Вы работаете с Битрикс?\n-...",280437919
4,485,2023-06-06 12:30:00,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,486,2023-06-06 12:41:38,"Здравствуйте, вроде 10 мышек разыгрывали, по-м...",387550114
5,485,2023-06-06 12:30:00,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,487,2023-06-06 14:01:35,Большое спасибо 1T Sprint за возможность участ...,752917537
6,485,2023-06-06 12:30:00,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,488,2023-06-06 14:59:33,"Спасибо за конкурс, приятно вспомнить с чего в...",699485614
7,383,2023-04-17 13:48:06,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,384,2023-04-17 16:33:48,,282137570
8,383,2023-04-17 13:48:06,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,385,2023-04-17 16:34:05,,282137570
9,383,2023-04-17 13:48:06,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,386,2023-04-17 19:01:47,Это был лучший офлайн)\nЭто было не реально кл...,25359617


### Почти все. Осталось только к id пользователя ('from_id'), оставившего комментарий, добавить полное имя ('first_name' + 'last_name').
### Запишем значения колонки 'comment_from_id' в отдельную переменную 'df_id_users':

In [13]:
df_id_users = df_all.comment_from_id
df_id_users

0      30472918
1      30472918
2     354733385
3     280437919
4     387550114
5     752917537
6     699485614
7     282137570
8     282137570
9      25359617
10    282137570
11    282137570
12    215426617
13    778096076
14    188085295
15    615150448
16    769557713
17    212331310
18     30472918
19      4692297
20    215426617
21    215426617
22    744813661
23     36054864
Name: comment_from_id, dtype: int64

### Важно! Информацию о пользователях сайт vk.com выдает с трудом, поэтому, если код ниже выдал ошибку, пожалуйста, запустите исполнение ячейки еще раз.
### Информацию о пользователях получим с помощью метода 'users.get'.

In [14]:
df_users = []
for comment_from_id in df_id_users:
    response = requests.get('https://api.vk.com/method/users.get',
                            params={
                                'access_token': token,
                                'user_ids': comment_from_id,
                                'v': version
                                }
                            )
    data = response.json()['response']
    df_users.extend(data)
df_users

[{'id': 30472918,
  'first_name': 'Nadezhda',
  'last_name': 'Lakhtina',
  'can_access_closed': True,
  'is_closed': False},
 {'id': 30472918,
  'first_name': 'Nadezhda',
  'last_name': 'Lakhtina',
  'can_access_closed': True,
  'is_closed': False},
 {'id': 354733385,
  'first_name': 'Dasha',
  'last_name': 'Novikova',
  'can_access_closed': True,
  'is_closed': False},
 {'id': 280437919,
  'first_name': 'Darya',
  'last_name': 'Prokopova',
  'can_access_closed': True,
  'is_closed': False},
 {'id': 387550114,
  'first_name': 'Vsyareklama',
  'last_name': 'Gsalavata',
  'can_access_closed': True,
  'is_closed': False},
 {'id': 752917537,
  'first_name': 'Evdokia',
  'last_name': 'Bekker',
  'can_access_closed': True,
  'is_closed': False},
 {'id': 699485614,
  'first_name': 'Nadia',
  'last_name': 'Lohengrin',
  'can_access_closed': True,
  'is_closed': False},
 {'id': 282137570,
  'first_name': 'Karina',
  'last_name': 'Chernitsova',
  'can_access_closed': True,
  'is_closed': False},

In [15]:
len(df_users)

24

### Вытянем информацию о полных именах пользователей по их 'id':

In [16]:
all_users_df = []
list_from_id, list_full_name = [], []
for user in df_users:
    list_from_id.append(user['id'])
    list_full_name.append(user['first_name'] + ' ' + user['last_name'])

all_users_df = pd.concat([pd.Series(list_from_id), pd.Series(list_full_name)], axis=1)
all_users_df.columns = ['comment_writer_id', 'writer_full_name']
all_users_df

,comment_writer_id,writer_full_name
0,30472918,Nadezhda Lakhtina
1,30472918,Nadezhda Lakhtina
2,354733385,Dasha Novikova
3,280437919,Darya Prokopova
4,387550114,Vsyareklama Gsalavata
5,752917537,Evdokia Bekker
6,699485614,Nadia Lohengrin
7,282137570,Karina Chernitsova
8,282137570,Karina Chernitsova
9,25359617,Denis Reshetnikov


### Смержим таблицы со всеми комментариями и полными именами пользователей по ключу 'id'.

In [17]:
df = df_all.merge(all_users_df, left_index=True, right_index=True, how='inner')
df

,post_id,date_public,text_post,comment_id,comment_date,comment_text,comment_from_id,comment_writer_id,writer_full_name
0,554,2023-07-14 13:04:05,Заключительная часть советов от участников раз...,555,2023-07-14 13:13:25,"Если ты работать вздумал,\nРазвернись и выпей ...",30472918,30472918,Nadezhda Lakhtina
1,554,2023-07-14 13:04:05,Заключительная часть советов от участников раз...,556,2023-07-14 14:03:42,"Если ты командной рулишь, \nМенеджер- руководи...",30472918,30472918,Nadezhda Lakhtina
2,531,2023-07-01 12:30:00,"7 полезных сервисов для тех, кто работает с те...",536,2023-07-04 16:19:27,"Классная подборка. Рады, что вы пользуетесь на...",354733385,354733385,Dasha Novikova
3,499,2023-06-11 12:30:00,Чего не стоит делать на собеседовании \n \nВы ...,500,2023-06-11 12:44:51,"Из последнего: \n""- Вы работаете с Битрикс?\n-...",280437919,280437919,Darya Prokopova
4,485,2023-06-06 12:30:00,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,486,2023-06-06 12:41:38,"Здравствуйте, вроде 10 мышек разыгрывали, по-м...",387550114,387550114,Vsyareklama Gsalavata
5,485,2023-06-06 12:30:00,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,487,2023-06-06 14:01:35,Большое спасибо 1T Sprint за возможность участ...,752917537,752917537,Evdokia Bekker
6,485,2023-06-06 12:30:00,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,488,2023-06-06 14:59:33,"Спасибо за конкурс, приятно вспомнить с чего в...",699485614,699485614,Nadia Lohengrin
7,383,2023-04-17 13:48:06,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,384,2023-04-17 16:33:48,,282137570,282137570,Karina Chernitsova
8,383,2023-04-17 13:48:06,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,385,2023-04-17 16:34:05,,282137570,282137570,Karina Chernitsova
9,383,2023-04-17 13:48:06,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,386,2023-04-17 19:01:47,Это был лучший офлайн)\nЭто было не реально кл...,25359617,25359617,Denis Reshetnikov


### Удалим ненужный, дублирующийся столбец 'comment_from_id'

In [18]:
df.drop(['comment_from_id'], axis=1 , inplace= True )
df

,post_id,date_public,text_post,comment_id,comment_date,comment_text,comment_writer_id,writer_full_name
0,554,2023-07-14 13:04:05,Заключительная часть советов от участников раз...,555,2023-07-14 13:13:25,"Если ты работать вздумал,\nРазвернись и выпей ...",30472918,Nadezhda Lakhtina
1,554,2023-07-14 13:04:05,Заключительная часть советов от участников раз...,556,2023-07-14 14:03:42,"Если ты командной рулишь, \nМенеджер- руководи...",30472918,Nadezhda Lakhtina
2,531,2023-07-01 12:30:00,"7 полезных сервисов для тех, кто работает с те...",536,2023-07-04 16:19:27,"Классная подборка. Рады, что вы пользуетесь на...",354733385,Dasha Novikova
3,499,2023-06-11 12:30:00,Чего не стоит делать на собеседовании \n \nВы ...,500,2023-06-11 12:44:51,"Из последнего: \n""- Вы работаете с Битрикс?\n-...",280437919,Darya Prokopova
4,485,2023-06-06 12:30:00,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,486,2023-06-06 12:41:38,"Здравствуйте, вроде 10 мышек разыгрывали, по-м...",387550114,Vsyareklama Gsalavata
5,485,2023-06-06 12:30:00,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,487,2023-06-06 14:01:35,Большое спасибо 1T Sprint за возможность участ...,752917537,Evdokia Bekker
6,485,2023-06-06 12:30:00,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,488,2023-06-06 14:59:33,"Спасибо за конкурс, приятно вспомнить с чего в...",699485614,Nadia Lohengrin
7,383,2023-04-17 13:48:06,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,384,2023-04-17 16:33:48,,282137570,Karina Chernitsova
8,383,2023-04-17 13:48:06,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,385,2023-04-17 16:34:05,,282137570,Karina Chernitsova
9,383,2023-04-17 13:48:06,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,386,2023-04-17 19:01:47,Это был лучший офлайн)\nЭто было не реально кл...,25359617,Denis Reshetnikov


### Ура! Наша таблица готова! Запишем ее в файл *.csv.

In [19]:
df.to_csv('total_comments.csv')